In [1]:
import pandas as pd
import numpy as np
from numpy import arccos, clip
from scipy.stats import linregress

import warnings
warnings.filterwarnings('ignore')


In [2]:
df_original = pd.read_csv('../data_cleaning/data/imputed_data.csv')
df = df_original.copy()

In [3]:
df.rename(columns={'person': 'participant'}, inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.sort_values(by=['timestamp'], ascending=True, inplace=True)
df

,timestamp,heart_rate,x_coordinate,y_coordinate,pupil_diameter_mm,iris_diameter_mm,pupil_iris_ratio,genre,participant
4116,2025-06-05 16:25:55,62.0,409.206897,194.000000,2.602941,11.8,0.220588,comedy,kenji
4117,2025-06-05 16:25:56,61.5,413.433333,194.800000,2.602941,11.8,0.220588,comedy,kenji
4118,2025-06-05 16:25:59,60.0,397.333333,191.000000,2.602941,11.8,0.220588,comedy,kenji
4119,2025-06-05 16:26:01,60.0,397.166667,190.600000,2.602941,11.8,0.220588,comedy,kenji
4120,2025-06-05 16:26:04,60.0,391.000000,189.333333,2.602941,11.8,0.220588,comedy,kenji
...,...,...,...,...,...,...,...,...,...
4111,2025-06-07 13:16:09,68.0,981.333333,434.200000,0.819444,11.8,0.069444,horror,clara
4112,2025-06-07 13:16:10,68.0,980.400000,430.200000,0.819444,11.8,0.069444,horror,clara
4113,2025-06-07 13:16:11,68.0,969.772727,415.409091,0.819444,11.8,0.069444,horror,clara
4114,2025-06-07 13:16:12,68.0,970.083333,417.875000,0.819444,11.8,0.069444,horror,clara


## 1. Head movement features

In [4]:
# Head displacement per frame
df['dx'] = df.groupby('participant')['x_coordinate'].diff()
df['dy'] = df.groupby('participant')['y_coordinate'].diff()
df['head_displacement'] = np.sqrt(df['dx']**2 + df['dy']**2)
df['head_displacement'] = df['head_displacement'].fillna(0)

In [5]:
# Head velocity per frame
df['dt'] = df.groupby('participant')['timestamp'].diff().dt.total_seconds()
df['head_velocity'] = df['head_displacement'] / df['dt']
df['head_velocity'] = df['head_velocity'].fillna(0)

In [6]:
# Head direction change rate

# Vector between t-1 and t
df['dx_prev'] = df.groupby('participant')['dx'].shift()
df['dy_prev'] = df.groupby('participant')['dy'].shift()

# Dot product and norms
dot = df['dx'] * df['dx_prev'] + df['dy'] * df['dy_prev']
norm_product = np.sqrt((df['dx']**2 + df['dy']**2) * (df['dx_prev']**2 + df['dy_prev']**2))
df['cos_theta'] = dot / norm_product
df['cos_theta'] = clip(df['cos_theta'], -1.0, 1.0)  # Avoid numerical issues

# Angle in radians
df['angle_change'] = arccos(df['cos_theta'])

# Use rolling sum or mean of angle changes as direction change rate
df['head_direction_change_rate'] = df.groupby('participant')['angle_change'].rolling(window=10, min_periods=1).mean().reset_index(level=0, drop=True)

df['head_direction_change_rate'] = df['head_direction_change_rate'].fillna(0)


In [7]:
# Head stability 
df['head_x_std'] = df.groupby('participant')['x_coordinate'].rolling(window=10, min_periods=1).std().reset_index(level=0, drop=True)
df['head_y_std'] = df.groupby('participant')['y_coordinate'].rolling(window=10, min_periods=1).std().reset_index(level=0, drop=True)
df['head_stability'] = (df['head_x_std'] + df['head_y_std']) / 2
df['head_stability'] = df['head_stability'].fillna(0)

In [8]:
# Centered coordinates
mean_x = df.groupby('participant')['x_coordinate'].transform('mean')
mean_y = df.groupby('participant')['y_coordinate'].transform('mean')
df['centered_x'] = df['x_coordinate'] - mean_x
df['centered_y'] = df['y_coordinate'] - mean_y


In [9]:
df.drop(columns=['x_coordinate', 'y_coordinate', 'dx', 'dy', 'dt', 'dx_prev', 'dy_prev', 'cos_theta', 'angle_change', 'head_stability', 'head_x_std', 'head_y_std'], inplace=True)

In [10]:
def reorder_cols(df): 
    df.sort_values(by=['timestamp'], ascending=True, inplace=True)

    # Get the current column order
    cols = list(df.columns)
    
    # Move 'participant' to 2nd and 'genre' to last
    cols.remove('participant')
    cols.remove('genre')
    new_order = [cols[0], 'participant'] + cols[1:] + ['genre']
    
    # Reorder the DataFrame
    return df[new_order]

df = reorder_cols(df)
df

,timestamp,participant,heart_rate,pupil_diameter_mm,iris_diameter_mm,pupil_iris_ratio,head_displacement,head_velocity,head_direction_change_rate,centered_x,centered_y,genre
4116,2025-06-05 16:25:55,kenji,62.0,2.602941,11.8,0.220588,0.000000,0.000000,0.000000,67.367276,-42.291024,comedy
4117,2025-06-05 16:25:56,kenji,61.5,2.602941,11.8,0.220588,4.301484,4.301484,0.000000,71.593713,-41.491024,comedy
4118,2025-06-05 16:25:59,kenji,60.0,2.602941,11.8,0.220588,16.542370,5.514123,3.096881,55.493713,-45.291024,comedy
4119,2025-06-05 16:26:01,kenji,60.0,2.602941,11.8,0.220588,0.433333,0.216667,2.020552,55.327046,-45.691024,comedy
4120,2025-06-05 16:26:04,kenji,60.0,2.602941,11.8,0.220588,6.295413,2.098471,1.671507,49.160379,-46.957691,comedy
...,...,...,...,...,...,...,...,...,...,...,...,...
4111,2025-06-07 13:16:09,clara,68.0,0.819444,11.8,0.069444,2.142688,2.142688,1.021154,56.031877,-62.891067,horror
4112,2025-06-07 13:16:10,clara,68.0,0.819444,11.8,0.069444,4.107446,4.107446,1.210504,55.098543,-66.891067,horror
4113,2025-06-07 13:16:11,clara,68.0,0.819444,11.8,0.069444,18.212905,18.212905,1.249884,44.471271,-81.681976,horror
4114,2025-06-07 13:16:12,clara,68.0,0.819444,11.8,0.069444,2.485394,2.485394,1.514270,44.781877,-79.216067,horror


## 2. Time Domain Features

In [11]:
# Define features and window size
signals = ['heart_rate', 'centered_x', 'centered_y', 'pupil_diameter_mm', 'iris_diameter_mm', 'pupil_iris_ratio']
window = '2s'

# Ensure timestamp is datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Function for custom features
def compute_features(group):
    group = group.set_index('timestamp').sort_index()

    for signal in signals:
        roll = group[signal].rolling(window=window, min_periods=2)

        # Basic stats
        group[f'td_{signal}_mean'] = roll.mean()
        group[f'td_{signal}_std'] = roll.std()
        group[f'td_{signal}_min'] = roll.min()
        group[f'td_{signal}_max'] = roll.max()
        group[f'td_{signal}_range'] = group[f'td_{signal}_max'] - group[f'td_{signal}_min']

        # Slope (linear regression over rolling window)
        group[f'td_{signal}_slope'] = roll.apply(
            lambda x: linregress(range(len(x)), x)[0] if len(x) > 1 else np.nan, raw=False
        )

        # Mean absolute change
        group[f'td_{signal}_mean_abs_change'] = roll.apply(
            lambda x: np.mean(np.abs(np.diff(x))) if len(x) > 1 else np.nan,
            raw=True
        )

        # Second-order difference mean 
        group[f'td_{signal}_second_order_diff_mean'] = roll.apply(
            lambda x: np.mean(np.abs(np.diff(x, n=2))) if len(x) > 2 else np.nan,
            raw=True
        )

    return group.reset_index()

# Apply per participant
df = df.groupby('participant', group_keys=False).apply(compute_features)

In [12]:
df.sort_values(by=['timestamp'], ascending=True, inplace=True)

In [13]:
# Backward fill because first one or two rows could be NaN
df = reorder_cols(df)
td_feature_columns = [col for col in df.columns if col.startswith('td_')]
df[td_feature_columns] = df[td_feature_columns].fillna(method='bfill').fillna(0)

In [14]:
df

,timestamp,participant,heart_rate,pupil_diameter_mm,iris_diameter_mm,pupil_iris_ratio,head_displacement,head_velocity,head_direction_change_rate,centered_x,...,td_iris_diameter_mm_second_order_diff_mean,td_pupil_iris_ratio_mean,td_pupil_iris_ratio_std,td_pupil_iris_ratio_min,td_pupil_iris_ratio_max,td_pupil_iris_ratio_range,td_pupil_iris_ratio_slope,td_pupil_iris_ratio_mean_abs_change,td_pupil_iris_ratio_second_order_diff_mean,genre
0,2025-06-05 16:25:55,kenji,62.0,2.602941,11.8,0.220588,0.000000,0.000000,0.000000,67.367276,...,0.0,0.220588,0.000000,0.220588,0.220588,0.000000,0.000000,0.000000,0.0,comedy
1,2025-06-05 16:25:56,kenji,61.5,2.602941,11.8,0.220588,4.301484,4.301484,0.000000,71.593713,...,0.0,0.220588,0.000000,0.220588,0.220588,0.000000,0.000000,0.000000,0.0,comedy
2,2025-06-05 16:25:59,kenji,60.0,2.602941,11.8,0.220588,16.542370,5.514123,3.096881,55.493713,...,0.0,0.216246,0.006141,0.211904,0.220588,0.008684,-0.008684,0.008684,0.0,comedy
3,2025-06-05 16:26:01,kenji,60.0,2.602941,11.8,0.220588,0.433333,0.216667,2.020552,55.327046,...,0.0,0.216246,0.006141,0.211904,0.220588,0.008684,-0.008684,0.008684,0.0,comedy
4,2025-06-05 16:26:04,kenji,60.0,2.602941,11.8,0.220588,6.295413,2.098471,1.671507,49.160379,...,0.0,0.216246,0.006141,0.211904,0.220588,0.008684,-0.008684,0.008684,0.0,comedy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4111,2025-06-07 13:16:09,clara,68.0,0.819444,11.8,0.069444,2.142688,2.142688,1.021154,56.031877,...,0.0,0.069444,0.000000,0.069444,0.069444,0.000000,0.000000,0.000000,0.0,horror
4112,2025-06-07 13:16:10,clara,68.0,0.819444,11.8,0.069444,4.107446,4.107446,1.210504,55.098543,...,0.0,0.069444,0.000000,0.069444,0.069444,0.000000,0.000000,0.000000,0.0,horror
4113,2025-06-07 13:16:11,clara,68.0,0.819444,11.8,0.069444,18.212905,18.212905,1.249884,44.471271,...,0.0,0.069444,0.000000,0.069444,0.069444,0.000000,0.000000,0.000000,0.0,horror
4114,2025-06-07 13:16:12,clara,68.0,0.819444,11.8,0.069444,2.485394,2.485394,1.514270,44.781877,...,0.0,0.069444,0.000000,0.069444,0.069444,0.000000,0.000000,0.000000,0.0,horror


## 3. Higher-Order Stats for Eye-Tracking


In [15]:
# skewness, kurtosis (for pupil_diameter_mm, iris_diameter_mm, pupil_iris_ratio)

## 4. Frequency-Domain Features


In [16]:
# Use FFT on heart rate and pupil signals:
# Power in 4 frequency bands:
# 0–0.2Hz (slow trend)
# 0.2–0.4Hz
# 0.4–0.6Hz
# 0.6–1.0Hz
# 
# 
# → For each band:
# band_power
# band_dominant_freq
# weighted_avg_freq
# spectral_entropy (total signal complexity)

## 5. Nonlinear Irregularity 

In [17]:
# For heart_rate, pupil_diameter_mm: approximate_entropy, sample_entropy

## 6. Categorical + Temporal Patterns


In [18]:
# Discretize numerical features into buckets (low, normal, high) using per-participant z-score ranges

# Extract temporal succession patterns of these buckets over time, e.g.:
# "high → high → normal" = stable high
# "low → normal → high" = rising
# count of stable/increasing/decreasing trends


## 7. Per-Person Normalization (Maybe not final step?!)

In [19]:
# Normalize each signal (z-score) per participant (critical for generalization to new test subject)


## 8. Clustering?